In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# %pip install -e .

HERE = Path.cwd().resolve()
REPO = HERE.parent
SRC  = REPO / "src"

# make 'boreas' importable from source tree
if (SRC / "boreas").exists():
    sys.path.insert(0, str(SRC))

from boreas import ModelParams, MassLoss, Fractionation
from boreas.flow_solutions import FlowSolutions as FS

try:
    from plots import Plots # if plots/ has an init.py file
except ImportError:
    from plots.plots import Plots

#### Loader for M-R .ddat files

In [ ]:
class ModelDataLoader:
    def __init__(self, base_path: Path, params: ModelParams):
        self.base_path = Path(base_path)
        self.params = params

    def load_single_ddat_file(self, ddat_filename):
        """
        Load a single .ddat file by name and return arrays for mass, radius, and temperature.
        """
        p = self.base_path / ddat_filename
        
        rows = []
        
        with p.open("r") as f:
            for line in f:
                s = line.strip()
                if not s or s.startswith("#"):
                    continue
                row = [float(x) for x in s.split()]
                if row[1] == -1.0:
                    continue
                rows.append(row)
                
        if not rows:
            raise ValueError(f"No valid rows in {p}")

        data = np.array(rows, dtype=float)
        mass_me     = data[:, 0]
        radius_re   = data[:, 1]
        teq_k       = data[:, 2]
        mass_g      = mass_me  * self.params.mearth
        radius_c    = radius_re* self.params.rearth
        return mass_g, radius_c, teq_k

#### Grids and composition: H₂ + H₂O only

In [ ]:
params  = ModelParams() # do this before setting the X_*, otherwise they will be reset

base    = Path("/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/paper") # path of M-R .ddat files
out_dir = Path("/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/paper_rerun/XUV_filter") # where to save the re-run

params.set_composition({"H2": 0., "H2O": 1.}, auto_normalize=True)
ddat_file = "3H2O_superEarth.ddat"
csv_path = HERE / "T-FXUV.csv" # Pierlou data

mass_loss     = MassLoss(params)
fractionation = Fractionation(params)
loader        = ModelDataLoader(base, params)

mass, radius, Teq = loader.load_single_ddat_file(ddat_file)
print(mass.shape, radius.shape, Teq.shape)

#### FXUV

In [ ]:
df_fxuv  = pd.read_csv(csv_path)

if "teq" not in df_fxuv.columns or "fxuv" not in df_fxuv.columns:
    raise ValueError("CSV must have columns 'Teq' and 'FXUV'.")
    
col_T = "teq"
col_F = "fxuv"
if col_T not in df_fxuv.columns or col_F not in df_fxuv.columns:
    # try some common variants
    ren = {}
    for c in df_fxuv.columns:
        lc = c.strip().lower()
        if lc in ("teq","teq_k","temperature","eq_temp"): ren[c] = col_T
        if lc in ("fxuv","f_xuv","xuv_flux"):             ren[c] = col_F
    df_fxuv = df_fxuv.rename(columns=ren)
    assert col_T in df_fxuv.columns and col_F in df_fxuv.columns, \
        f"CSV must have '{col_T}' and '{col_F}' columns."

# Optional: de-duplicate within each Teq while preserving order
def unique_in_order(x): 
    seen=set(); out=[]
    for v in x:
        if v not in seen: seen.add(v); out.append(v)
    return out

# Handle floating Teq binning: round to e.g. nearest 1 K to stabilize grouping
df_fxuv["Teq_key"] = df_fxuv[col_T].round(0)

# Build: Teq_key -> np.array([FXUV_1, FXUV_2, ...])
fxuv_map = (
    df_fxuv
    .groupby("Teq_key", sort=True)[col_F]
    .apply(lambda s: np.array(unique_in_order(list(s.astype(float))), dtype=float))
    .to_dict()
)

# Helper: get FXUVs for a Teq (exact rounded match; fallback to nearest key)
def fxuvs_for_teq(teq_val: float, keys=None):
    if keys is None: keys = np.array(sorted(fxuv_map.keys()), dtype=float)
    k = float(np.round(teq_val, 0))
    if k in fxuv_map:
        return fxuv_map[k]
    # fallback to nearest available Teq
    nearest = float(keys[np.argmin(np.abs(keys - k))])
    return fxuv_map[nearest]

#### Run the model grouped by unique Teq (reusing FXUVs)

In [ ]:
rows = []

DIAG_COLS = ["EL_min_abs_f", "EL_Rbest", "EL_Rbest_over_Rp", "skip_reason"]
COMP_KEYS = ["H2","H2O","O2","CO2","CO","CH4","N2","NH3","H2S","SO2","S2"]
def composition_payload(p):
    return {f"X_{k}": float(getattr(p, f"X_{k}")) for k in COMP_KEYS}

for T in np.unique(Teq):
    print(f"\n Running Teq = {T} K")
    mask = np.isclose(Teq, T)
    M, R, Tgrp = mass[mask], radius[mask], Teq[mask]
    
    for FXUV in fxuvs_for_teq(float(T)): # <-- values from T-FXUV.csv (Pierlou data)
        print(f"Running FXUV = {FXUV} erg/cm^2/s")
        params.update_param("FXUV", float(FXUV))
        
        ml  = mass_loss.compute_mass_loss_parameters(M, R, Tgrp)
        ml_ok = [rec for rec in ml if rec.get("regime") != "SKIPPED"]
        ml_skipped = [rec for rec in ml if rec.get("regime") == "SKIPPED"]
        
        fr  = fractionation.execute(ml_ok, mass_loss, allow_dynamic_light_major=True, forced_light_major="H", tol=1e-5, max_iter=100)
        
        comp = composition_payload(params)
        
        for rec in fr:
            row = {"FXUV": float(FXUV)}
            row.update(rec)    # <- bring in ALL keys from mass-loss + fractionation
            row.update(comp)   # <- composition snapshot
            for c in DIAG_COLS:
                row.setdefault(c, float("nan"))
            rows.append(row)

            
        # also record skipped with diagnostics
        for rec in ml_skipped:
            row = {"FXUV": float(FXUV)}
            row.update(rec)
            row.update(comp)
            for c in DIAG_COLS:
                row.setdefault(c, rec.get(c, float("nan")))
            rows.append(row)

df_results = pd.DataFrame(rows)
# ensure all columns present even if missing in some rows
all_cols = sorted(set().union(*(r.keys() for r in rows)))
df_results = df_results.reindex(columns=all_cols)

csv_name = Path(ddat_file).with_suffix(".csv").name
csv_path = out_dir / csv_name
df_results.to_csv(csv_path, index=False)
print("wrote", csv_path)